In [2]:
example = """O....#....
O.OO#....#
.....##...
OO.#O....O
.O.....O#.
O.#..O.#.#
..O..#O..O
.......O..
#....###..
#OO..#...."""

In [3]:
with open('./data/Day 14/input.txt', 'r') as file:
    data = file.read()

In [4]:
import numpy as np
def parse_rocks(rock_str):
    temp_field = []
    for line in rock_str.splitlines():
        temp_field.append(list(line))
    return np.array(temp_field)

In [70]:
def check_movement(rock_array, loc, direction):
    if direction == "North":
        if loc[0] == 0:
            return rock_array
        if rock_array[loc] == 'O' and rock_array[(loc[0]-1, loc[1])] == '.':
            rock_array[(loc[0] - 1, loc[1])] = "O"
            rock_array[loc] = '.'
            rock_array = check_movement(rock_array, (loc[0] - 1, loc[1]), direction)
    elif direction == "South":
        if loc[0] == rock_array.shape[0] -1:
            return rock_array
        if rock_array[loc] == "O" and rock_array[(loc[0] + 1, loc[1])] == ".":
            rock_array[(loc[0] + 1, loc[1])] = "O"
            rock_array[loc] = "."
            rock_array = check_movement(rock_array, (loc[0] + 1, loc[1]), direction)
    elif direction == "East":
        if loc[1] == rock_array.shape[1] - 1:
            return rock_array
        if rock_array[loc] == "O" and rock_array[(loc[0], loc[1] + 1)] == ".":
            rock_array[(loc[0], loc[1] + 1)] = "O"
            rock_array[loc] = "."
            rock_array = check_movement(rock_array, (loc[0], loc[1] + 1), direction)
    elif direction == "West":
        if loc[1] == 0:
            return rock_array
        if rock_array[loc] == "O" and rock_array[(loc[0], loc[1] - 1)] == ".":
            rock_array[(loc[0], loc[1] - 1)] = "O"
            rock_array[loc] = "."
            rock_array = check_movement(rock_array, (loc[0], loc[1] - 1), direction)
    return rock_array

In [62]:
def move_north(rock_array):
    """ "Moves all 0 to the top until it hits a O or a #"""
    for i in range(rock_array.shape[0]):
        for j in range(rock_array.shape[1]):
            check_movement(rock_array, (i,j), "North")
    return rock_array

In [73]:
def move_cycle(rock_array):
    for i in range(rock_array.shape[0]):
        for j in range(rock_array.shape[1]):
            rock_array = check_movement(rock_array, (i, j), "North")
    for i in range(rock_array.shape[0]):
        for j in range(rock_array.shape[1]):
            rock_array = check_movement(rock_array, (i, j), "West")
    for i in range(rock_array.shape[0]-1, -1, -1):
        for j in range(rock_array.shape[1]-1, -1, -1):
            rock_array = check_movement(rock_array, (i, j), "South")
    for i in range(rock_array.shape[0]-1, -1, -1):
        for j in range(rock_array.shape[1]-1, -1, -1):
            rock_array = check_movement(rock_array, (i, j), "East")
    return rock_array

In [74]:
def score_rocks(rock_array):
    locations_row = np.where(rock_array == "O")[0]
    distance_top = rock_array.shape[0] - locations_row
    return sum(distance_top)

# Part 1

In [75]:
rock_example_array = parse_rocks(example)
example_moved_rocks = move_north(rock_example_array.copy())
score_rocks(example_moved_rocks)

136

In [76]:
rock_array = parse_rocks(data)
moved_rocks = move_north(rock_array.copy())
score_rocks(moved_rocks)

109833

# Part 2

In [77]:
rock_array

array([['.', 'O', '.', ..., 'O', '.', '#'],
       ['.', '.', 'O', ..., '.', 'O', '.'],
       ['.', '.', '.', ..., '.', '.', '.'],
       ...,
       ['O', '.', '.', ..., 'O', '#', '.'],
       ['#', '.', '.', ..., '#', '.', '.'],
       ['.', 'O', '.', ..., '.', '.', '.']], dtype='<U1')

In [78]:
from tqdm import tqdm

In [111]:
def find_cycle(rock_array):
    previous_rocks = [rock_array.copy()]
    cycle_found = False
    for cycle in tqdm(range(1000000000)):
        current_rocks = move_cycle(previous_rocks[-1].copy())
        for index, rock in enumerate(previous_rocks):
            if np.array_equal(rock, current_rocks):
                print(f"Cycle: {cycle}; Index {index}")
                cycle_found = True
                break
        previous_rocks.append(current_rocks)
        if cycle_found:
            return cycle, index, previous_rocks

In [112]:
cycle, index, rocks = find_cycle(rock_array)

  0%|          | 0/1000000000 [00:00<?, ?it/s]

  0%|          | 169/1000000000 [00:29<48768:20:12,  5.70it/s]

Cycle: 169; Index 142


In [113]:
position = (1000000000 - index) % (cycle - index + 1) + index
score_rocks(rocks[position])

99875

In [114]:
example_cycle, example_index, example_rocks = find_cycle(rock_example_array)

  0%|          | 9/1000000000 [00:00<479:02:14, 579.87it/s]

Cycle: 9; Index 3


In [115]:
position = (1000000000 - example_index) % (example_cycle - example_index + 1) + example_index
score_rocks(example_rocks[position])

64